In [7]:
#Installing and importing major needs
!pip install openai
!pip install transformers
import json
import numpy as np
import pandas as pd
import tensorflow as tf 
from matplotlib import pyplot as plt
from google.colab import drive
import os
!pip install -q -U tensorflow-text ## install the package for NLP tasks in tensorflow
import tensorflow_hub as hub
import tensorflow_text
import tensorflow as tf
from tensorflow import keras
import pickle
drive.mount('/content/drive')
%matplotlib inline


def unpickle_vars(fileName, env):
    with open(fileName, 'rb') as f:
        d = pickle.load(f)
        env.update(d)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
bert_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

bert_layers = 12
bert_units = 768
bert_heads = 12

bert_encoder = f'https://tfhub.dev/tensorflow/bert_en_uncased_L-{bert_layers}_H-{bert_units}_A-{bert_heads}/4'

In [ ]:
max_length = 512
preprocessor = hub.load(bert_preprocess)
encoder = hub.KerasLayer(bert_encoder, trainable=False)

def bert_textvect(x):
  input = keras.layers.Input(shape=(), dtype=tf.string)
  tokenized_input = hub.KerasLayer(preprocessor.tokenize)(input)
  bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs, arguments=dict(seq_length=max_length))
  output = bert_pack_inputs([tokenized_input])
  model = keras.Model(input, output)
  result = model.predict(x)
  return result


def bert_features(x):
  inputs = dict(
    input_word_ids=keras.layers.Input(shape=(max_length,), dtype=tf.int32),
    input_mask=keras.layers.Input(shape=(max_length,), dtype=tf.int32),
    input_type_ids=keras.layers.Input(shape=(max_length,), dtype=tf.int32),
  )

  output = encoder(inputs)['sequence_output'][:, 0, :]
  model = keras.Model(inputs, output)
  return model.predict(x)


In [10]:
unpickle_vars('/content/drive/MyDrive/HoDL Project - Group 7/yelp_dataset/1_5Star_Reviews.pkl',globals())

y_train_dumm = pd.get_dummies(np.asarray(y_train))
y_test_dumm = pd.get_dummies(np.asarray(y_test))

In [13]:
hidden_units = 64 

# Input
input = keras.Input(shape=(bert_units, ))

# Classification Head
x = keras.layers.Dense(hidden_units,activation='relu')(input)
x = keras.layers.Dense(hidden_units,activation='relu')(x)
#x = keras.layers.Dropout(0.1)(x)
output = keras.layers.Dense(2, activation='softmax')(x)

# Model
model = keras.Model(input, output)

model.summary()

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 768)]             0         
                                                                 
 dense (Dense)               (None, 64)                49216     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 53,506
Trainable params: 53,506
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(x=features_train, y=y_train_dumm, validation_data=(features_test, y_test_dumm),epochs=5,batch_size=64)

Epoch 1/5
345/345 [==============================] - 6s 11ms/step - loss: 0.1117 - accuracy: 0.9551 - val_loss: 0.0932 - val_accuracy: 0.9649
Epoch 2/5
345/345 [==============================] - 2s 5ms/step - loss: 0.0774 - accuracy: 0.9721 - val_loss: 0.0810 - val_accuracy: 0.9697
Epoch 3/5
345/345 [==============================] - 3s 7ms/step - loss: 0.0721 - accuracy: 0.9725 - val_loss: 0.0785 - val_accuracy: 0.9719
Epoch 4/5
345/345 [==============================] - 2s 6ms/step - loss: 0.0677 - accuracy: 0.9745 - val_loss: 0.0824 - val_accuracy: 0.9719
Epoch 5/5
345/345 [==============================] - 2s 5ms/step - loss: 0.0618 - accuracy: 0.9781 - val_loss: 0.0918 - val_accuracy: 0.9672


Model Evaluate

---



In [ ]:
testReview = ["Test"]
encoded_input = bert_features(bert_textvect(testReview))
model.predict(encoded_input)

NameError: ignored

In [22]:
result = []
for num,review in enumerate(features_test):
  result.append(model.predict(tf.expand_dims(review,0)))


In [24]:
result = np.asarray(result).squeeze()
pd.DataFrame.to_csv(pd.DataFrame({'Reviews':x_test,'Rating':y_test,'1 star P':result[:,0],'5 star P':result[:,1]}),'/content/drive/MyDrive/HoDL Project - Group 7/1-5_Results_test.csv')